Copyright ENEOS corporation as contributors to Matlantis contrib project

#  Notebook２_触媒表面での計算

# 目次

[必要なモジュールの読み込み](#必要なモジュールの読み込み)  
[吸着構造の計算](#吸着構造の計算)  
[復習：分子の読み込みとエネルギー計算](#復習：分子の読み込みとエネルギー計算)  
[結晶表面モデルの作成](#結晶表面モデルの作成)  
[①バルクの作成](#①バルクの作成)  
[Tips : Constraintについて](#Tips-:-Constraintについて)  
[②スラブの作成](#②スラブの作成)  
[③反応分子の配置](#③反応分子の配置)  
[吸着エネルギーの計算](#吸着エネルギーの計算)  
[Tips : Atomsオブジェクトについて](#tips-:-atomsオブジェクトについて)

## 必要なモジュールの読み込み
※notebook1で使用済のモジュールはここでimport  
　新規に使用するモジュールは適宜importします

In [1]:
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator
estimator = Estimator(model_version='v2.0.0') #version指定を推奨
calculator = ASECalculator(estimator)

import ase
from ase.visualize import view
from ase.build import surface, molecule
from ase.optimize import BFGS

import pathlib
import os
os.makedirs("output/", exist_ok=True)

# 吸着構造の計算  
吸着エネルギーや界面での反応を見たい場合にはスラブモデルが有効です。  
ここではPt(111)面上のCO吸着モデルの作成、CO吸着エネルギーの算出を目標に進めます。  
吸着エネルギーE(ads)は以下のエネルギーから求めます。  
`E(ads) = E(slab) + E(co) - E(co_on_slab)`  
必要な構造は以下の3つです。  
(1)CO分子単体(2)Ptスラブのみ(3)COがPtスラブに吸着したモデル  
本ノートブックでは(1)-(3)のモデルを順に作成し、計算を行います。  

# 復習：分子の読み込みとエネルギー計算
まずCO分子単体の計算を行います。

In [2]:
mol_CO = molecule("CO")
mol_CO.set_calculator(calculator)
mol_CO.pbc = False
opt = BFGS(mol_CO)
opt.run(fmax=0.01,steps=500)
v = view(mol_CO, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

      Step     Time          Energy         fmax
BFGS:    0 00:49:51      -11.481743        0.7964
BFGS:    1 00:49:51      -11.470749        1.8432
BFGS:    2 00:49:51      -11.484685        0.0505
BFGS:    3 00:49:52      -11.484686        0.0031


# 結晶表面モデルの作成

ここからが本題です。結晶表面モデルは  
**①バルク最適化**  
**②面の切り出しと真空層の決定、ユニットセル・原子の固定**  
**③反応分子の配置**   
といった手順で作成します。

## ①バルクの作成
まずMaterial Project( https://materialsproject.org/materials/mp-126/ )よりfcc-Ptのcif(結晶構造)ファイル(Pt_mp-126_conventional_standard.cif)をダウンロードし、inputフォルダに置きます。  
計算手法によって安定な格子定数が変化するため、はじめに格子定数の最適化を行います。  
ここでは、元の構造の対称性を保持する`FixSymmetry`を拘束条件として設定し、セルパラメータを含めた最適化を行うため、`UnitCellFiter`を用います。

In [3]:
from ase.constraints import UnitCellFilter, FixAtoms
from ase.spacegroup.symmetrize import FixSymmetry

bulk = ase.io.read("input/Pt_mp-126_conventional_standard.cif")
print(bulk.get_cell_lengths_and_angles())                    #構造緩和前の格子定数を確認
bulk.set_calculator(calculator)
bulk.set_constraint([FixSymmetry(bulk, adjust_cell=True,
                                    adjust_positions=True)]) #対称性を保ちながら原子座標、セルの大きさ両方を最適化
ucf = UnitCellFilter(bulk)
opt = BFGS(ucf)                                              #ucfを適用したPt_bulkを構造緩和
opt.run(fmax=0.01,steps=500)
bulk.wrap                                                    #原子がセル内に入るように再処理 
print(bulk.get_cell_lengths_and_angles())                    #構造緩和後の格子定数を確認
bulk.translate(-0.3)                                          #原子の座標を調整し、表面をきれいに出せるようにする

/home/jovyan/.local/lib/python3.7/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


[ 3.97677  3.97677  3.97677 90.      90.      90.     ]
      Step     Time          Energy         fmax
BFGS:    0 00:50:07      -21.807936        0.1512
BFGS:    1 00:50:08      -21.808818        0.1190
BFGS:    2 00:50:08      -21.810196        0.0051
[ 3.96667761  3.96667761  3.96667761 90.         90.         90.        ]


# Tips : Constraintについて
Constraintは構造に加える制約条件で、上記で設定した`FixSymmetry`以外にも`FixAtom`(原子座標を固定)など、複数の手法が用意されています。
Constraintの解除は以下のように行います。

In [4]:
bulk.set_constraint() #スラブ作成に必要な操作です。必ず実行してください。

この操作を行わない限り、構造をcopyなどした際にもconstraintが残るため、意図しない制約を加えてしまうことがあるので注意してください。

## ②スラブの作成
ASEでは表面の切り出しを簡単に行うことができます。  
Pt(111)は以下のように面を切り出すことができます。

In [5]:
bulk.set_positions(bulk.get_positions()-[0.5,0.5,0.5])      #少し座標をずらすことで綺麗に面切り出し可能
slab = surface(bulk,indices=(1,1,1),                        #切り出す面を指定
                         layers=4, vacuum=5, periodic=True) #真空層を表面の上下に5Åずつ設定           
slab = slab.repeat([2,2,1])                                 #4*4の面を作成
display(view(slab, viewer="ngl"))

作成したスラブは保存し、分子吸着用の変数にコピーします。

In [6]:
ase.io.write("output/PtSlab.json", slab) #作成したslabを保存
mol_on_slab = slab.copy() #分子吸着用の別の変数を作成

### 分子の移動・追加・削除
先ほど作成したスラブをセル下部に移動します。

In [7]:
for atom in mol_on_slab:
    atom.z -= 5 #slabをセル下部に配置するため全原子のz座標を移動
display(view(mol_on_slab, viewer="ngl"))

作成したスラブ上部にH2分子を追加します。

In [8]:
from ase.build import molecule, add_adsorbate
add_adsorbate(mol_on_slab, molecule('H2'), height=2.0, position=[8,4]) #分子の追加
display(view(mol_on_slab, viewer="ngl"))

削除は配列の要素番号を指定して行うことができます。

In [9]:
del mol_on_slab[-2:] #インデックスの最後から2原子(ここではH2)を指定して削除

他にも分子の編集には様々なメソッドが用意されているので、必要に応じてASEドキュメントを参照してください。

## ③反応分子の配置
H2の場合と同様にCOをPtスラブモデルに追加し、編集を行います。

In [10]:
add_adsorbate(mol_on_slab, molecule('CO'), height=3.0, position=[8,4]) #分子の追加
display(view(mol_on_slab, viewer="ngl"))

被覆率が低い場合ではCOはPt(111)のtop-siteに吸着することが実験的に知られています[1]。
[1] https://pubs.acs.org/doi/10.1021/jp002302t

今回は先行研究に倣いCO分子をtop-siteに近づけた初期構造を作成し、最適化を行うこととします。

In [11]:
for i in [-2,-1]:                  # CO分子をhcpサイト付近に移動
    mol_on_slab[i].position += [0.5,-0.5,0]    # CO分子をx方向に0.5Å,y方向に-0.5Å移動

可視化しながらCO分子の位置を確認し、COを平行移動して吸着位置を調整してください。

In [12]:
display(view(mol_on_slab, viewer="ngl"))

下から2層を固定して構造最適化を行います。
後程の吸着エネルギー計算のためにPtスラブのみでも同様の条件で最適化を行ってください。

In [13]:
c = FixAtoms(indices=[i for i,atom in enumerate(mol_on_slab) if  mol_on_slab[i].z < 4] ) # 条件文でスラブ下2層のPt原子を指定
mol_on_slab.set_constraint(c)
mol_on_slab.set_calculator(calculator)
opt = BFGS(mol_on_slab)
opt.run(fmax=0.01)

      Step     Time          Energy         fmax
BFGS:    0 00:51:12     -342.453406        0.9872
BFGS:    1 00:51:12     -342.475166        2.0914
BFGS:    2 00:51:12     -342.529088        1.1144
BFGS:    3 00:51:13     -342.567648        0.7216
BFGS:    4 00:51:13     -342.581965        0.6458
BFGS:    5 00:51:13     -342.601582        0.2482
BFGS:    6 00:51:14     -342.610766        0.4052
BFGS:    7 00:51:14     -342.621690        0.4669
BFGS:    8 00:51:14     -342.628140        0.3280
BFGS:    9 00:51:14     -342.632806        0.1536
BFGS:   10 00:51:15     -342.636879        0.2641
BFGS:   11 00:51:15     -342.643016        0.3977
BFGS:   12 00:51:15     -342.649754        0.3845
BFGS:   13 00:51:15     -342.654202        0.2089
BFGS:   14 00:51:16     -342.656590        0.0904
BFGS:   15 00:51:16     -342.658427        0.1418
BFGS:   16 00:51:16     -342.659870        0.1477
BFGS:   17 00:51:17     -342.660783        0.0956
BFGS:   18 00:51:17     -342.661268        0.0466
B

True

In [14]:
display(view(mol_on_slab, viewer="ngl"))

In [15]:
ase.io.write("output/CO_on_PtSlab.json", mol_on_slab)

In [16]:
## Pt_slabのみの最適化
c = FixAtoms(indices=[i for i,atom in enumerate(slab) if  slab[i].z < 4] ) # 条件文でスラブ下2層を指定
slab.set_constraint(c)
slab.set_calculator(calculator)
opt = BFGS(slab)
opt.run(fmax=0.01)

      Step     Time          Energy         fmax
BFGS:    0 00:51:37     -329.502679        0.2172
BFGS:    1 00:51:37     -329.530587        0.1430
BFGS:    2 00:51:38     -329.557011        0.0521
BFGS:    3 00:51:38     -329.558727        0.0431
BFGS:    4 00:51:38     -329.567056        0.0421
BFGS:    5 00:51:38     -329.571358        0.0297
BFGS:    6 00:51:39     -329.572781        0.0130
BFGS:    7 00:51:39     -329.572934        0.0022


True

# 吸着エネルギーの計算

最後にここまでで作成したモデルよりエネルギーを求め、吸着エネルギーを計算します。

In [17]:
slab.set_calculator(calculator)
mol_CO.set_calculator(calculator)
mol_on_slab.set_calculator(calculator)
Eads = slab.get_potential_energy() + mol_CO.get_potential_energy() - mol_on_slab.get_potential_energy()
print(Eads)

1.612190908177979


実験値は1.39 eV[2]ですが、吸着エネルギーが過大評価されています
吸着エネルギー計算はDFT計算でも汎関数によって大きくずれることが知られており、確認が重要となります。

[2]岩澤 康裕, 中村 潤児, 福井 賢一, 吉信 淳: "ベーシック 表面化学", 化学同人 (2010)

補正を入れる(DFTD3など)ことで実験値により近い値を得ることができることがあるので、事前に検討を行ってください。

# Tips : Atomsオブジェクトについて
ここまで扱ってきた `slab`などの分子情報はASEのAtomsオブジェクトとして管理されています。  
(1)分子の座標情報(2)Calculator(3)PBCの有無(4)セル(5)Constraint  
といった分子シミュレーションに関わる情報が一括で管理されています。  
これらの情報は

In [18]:
slab.get_positions()

array([[ 5.60972939e+00,  3.23877865e+00,  4.97619197e+00],
       [ 4.20729697e+00,  8.09694607e-01,  4.97619206e+00],
       [ 2.80486460e+00,  3.23877870e+00,  4.97619214e+00],
       [ 1.40243240e+00,  8.09694643e-01,  4.97619201e+00],
       [ 2.80486461e+00,  1.61938934e+00,  7.29292125e+00],
       [ 4.20729680e+00,  4.04847342e+00,  7.29292129e+00],
       [ 5.60972936e+00,  1.61938932e+00,  7.29292129e+00],
       [ 7.01216165e+00,  4.04847330e+00,  7.29292144e+00],
       [ 5.60972925e+00,  5.76598446e-08,  9.57756581e+00],
       [ 7.01216159e+00,  2.42908394e+00,  9.57756588e+00],
       [ 2.80486466e+00, -1.46624421e-07,  9.57756589e+00],
       [ 4.20729701e+00,  2.42908404e+00,  9.57756582e+00],
       [ 5.60972935e+00,  3.23877881e+00,  1.18942950e+01],
       [ 4.20729707e+00,  8.09694663e-01,  1.18942950e+01],
       [ 2.80486459e+00,  3.23877874e+00,  1.18942951e+01],
       [ 1.40243229e+00,  8.09694737e-01,  1.18942950e+01],
       [ 8.41459395e+00,  8.09694679e+00

などといったように確認することができます。  
PFPの基本機能であるEnergyやFoeceの出力も以下のように実行できます。

In [19]:
slab.get_forces() #Constraintが適用されている原子は力が0と出力される

array([[-2.38140830e-06,  1.04891597e-07, -2.07121952e-04],
       [ 5.92266702e-07, -5.41274815e-07, -2.09155518e-04],
       [ 6.84755685e-07, -4.12677322e-07, -2.11117304e-04],
       [-1.81777202e-06, -9.81615633e-07, -2.08890544e-04],
       [-7.68062703e-07,  5.92891530e-07,  2.17351987e-03],
       [ 1.23697588e-06, -3.81050993e-07,  2.17122232e-03],
       [-1.77165536e-07,  3.76907279e-07,  2.16961130e-03],
       [-1.94242498e-06,  1.99529977e-07,  2.16849246e-03],
       [ 1.43384318e-06, -7.42046569e-07, -2.17184330e-03],
       [ 4.53866009e-08, -4.16227794e-07, -2.16973063e-03],
       [-1.43176116e-06,  2.65776664e-06, -2.17215040e-03],
       [-1.49513644e-07, -1.97905899e-06, -2.17113132e-03],
       [-1.04834335e-06,  5.54025142e-07,  2.09257120e-04],
       [-5.56879546e-07,  1.70069215e-07,  2.10112610e-04],
       [ 1.38841201e-06, -1.19112814e-06,  2.10153076e-04],
       [ 1.26528232e-06,  8.75365356e-07,  2.09462801e-04],
       [ 3.84625010e-07,  2.64397739e-06

詳細は[ASE公式ドキュメント](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase.atoms)をご参照ください。